In [1]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import re
import string
from skimage import io, transform
import os
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [2]:
n_past = 5
n_future = 1

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
full_WP = pd.read_csv("./Full WP dataset.csv")

C:\Users\Louis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,8,9,14,25,131,132,133,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Data Preprcoessing

In [6]:
full_WP = full_WP.iloc[1:]

In [7]:
full_WP = full_WP.loc[full_WP['SEASON'].astype(int) >= 2000]

In [8]:
columns_features = ['ISO_TIME', 'SEASON', 'NAME', 'USA_LAT', 'USA_LON', 'DIST2LAND', 'USA_WIND', 'USA_PRES', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW', 'USA_R34_NW', 'USA_R50_NE', 'USA_R50_SE', 'USA_R50_SW', 'USA_R50_NW', 'USA_R64_NE', 'USA_R64_SE', 'USA_R64_SW', 'USA_R64_NW']

In [9]:
full_WP_modify = full_WP[columns_features].copy()

In [10]:
full_WP_modify = full_WP_modify.reset_index(drop=True)

In [11]:
full_WP_modify = full_WP_modify[full_WP_modify.NAME != 'NOT_NAMED']

In [12]:
full_WP_modify = full_WP_modify.reset_index(drop=True)

In [13]:
full_WP_modify = full_WP_modify[full_WP_modify.USA_LAT!=' ']

In [14]:
full_WP_modify = full_WP_modify.replace(r'^\s*$', -1 , regex=True)

In [15]:
full_WP_modify['USA_LAT'] = full_WP_modify['USA_LAT'].astype(float)

In [16]:
full_WP_modify['USA_LON'] = full_WP_modify['USA_LON'].astype(float)

In [17]:
full_WP_modify = full_WP_modify.reset_index(drop=True)

In [18]:
full_WP_modify['USA_LAT'].min()

1.3

In [19]:
full_WP_modify['USA_LAT'].max()

51.3

In [20]:
full_WP_modify['USA_LON'].min()

-180.0

In [21]:
full_WP_modify['USA_LON'].max()

180.0

In [22]:
full_WP_modify['USA_LON'][full_WP_modify[full_WP_modify.USA_LON < 0].index] = full_WP_modify['USA_LON'][full_WP_modify[full_WP_modify.USA_LON < 0].index] + 360

C:\Users\Louis\AppData\Local\Temp/ipykernel_10652/916726116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_WP_modify['USA_LON'][full_WP_modify[full_WP_modify.USA_LON < 0].index] = full_WP_modify['USA_LON'][full_WP_modify[full_WP_modify.USA_LON < 0].index] + 360


In [23]:
full_WP_modify['USA_LON'].max()

244.9

In [24]:
full_WP_modify['USA_LON'].min()

78.6

In [25]:
long_range = full_WP_modify.index[full_WP_modify['USA_LON'] <0].tolist()

In [26]:
len(long_range)

0

In [27]:
dict_remove = {}
for i in long_range:
    print(i)
    name = full_WP_modify.iloc[i,2]
    year = full_WP_modify.iloc[i,1]
    dict_remove[year] = name

In [28]:
dict_remove

{}

In [29]:
from sklearn.preprocessing import MinMaxScaler

scaler_lat = MinMaxScaler()
scaler_long = MinMaxScaler()
scaler_dist = MinMaxScaler()
scaler_wind = MinMaxScaler()
scaler_pres = MinMaxScaler()
scaler_r34_ne = MinMaxScaler()
scaler_r34_se = MinMaxScaler()
scaler_r34_sw = MinMaxScaler()
scaler_r34_nw = MinMaxScaler()
scaler_r50_ne = MinMaxScaler()
scaler_r50_se = MinMaxScaler()
scaler_r50_sw = MinMaxScaler()
scaler_r50_nw = MinMaxScaler()
scaler_r64_ne = MinMaxScaler()
scaler_r64_se = MinMaxScaler()
scaler_r64_sw = MinMaxScaler()
scaler_r64_nw = MinMaxScaler()

In [30]:
full_WP_modify[['USA_LAT']] = scaler_lat.fit_transform(full_WP_modify[['USA_LAT']])
full_WP_modify[['USA_LON']] = scaler_long.fit_transform(full_WP_modify[['USA_LON']])
full_WP_modify[['DIST2LAND']] = scaler_dist.fit_transform(full_WP_modify[['DIST2LAND']])
full_WP_modify[['USA_WIND']] = scaler_wind.fit_transform(full_WP_modify[['USA_WIND']])
full_WP_modify[['USA_PRES']] = scaler_pres.fit_transform(full_WP_modify[['USA_PRES']])
full_WP_modify[['USA_R34_NE']] = scaler_r34_ne.fit_transform(full_WP_modify[['USA_R34_NE']])
full_WP_modify[['USA_R34_SE']] = scaler_r34_se.fit_transform(full_WP_modify[['USA_R34_SE']])
full_WP_modify[['USA_R34_SW']] = scaler_r34_sw.fit_transform(full_WP_modify[['USA_R34_SW']])
full_WP_modify[['USA_R34_NW']] = scaler_r34_nw.fit_transform(full_WP_modify[['USA_R34_NW']])
full_WP_modify[['USA_R50_NE']] = scaler_r50_ne.fit_transform(full_WP_modify[['USA_R50_NE']])
full_WP_modify[['USA_R50_SE']] = scaler_r50_se.fit_transform(full_WP_modify[['USA_R50_SE']])
full_WP_modify[['USA_R50_SW']] = scaler_r50_sw.fit_transform(full_WP_modify[['USA_R50_SW']])
full_WP_modify[['USA_R50_NW']] = scaler_r50_nw.fit_transform(full_WP_modify[['USA_R50_NW']])
full_WP_modify[['USA_R64_NE']] = scaler_r64_ne.fit_transform(full_WP_modify[['USA_R64_NE']])
full_WP_modify[['USA_R64_SE']] = scaler_r64_se.fit_transform(full_WP_modify[['USA_R64_SE']])
full_WP_modify[['USA_R64_SW']] = scaler_r64_sw.fit_transform(full_WP_modify[['USA_R64_SW']])
full_WP_modify[['USA_R64_NW']] = scaler_r64_nw.fit_transform(full_WP_modify[['USA_R64_NW']])

In [31]:
for year in range(2000,2022):
     globals()['df_WP_%s' % year] = full_WP_modify[(full_WP_modify["SEASON"]==year)]

In [32]:
for year in range(2000,2022):
    globals()['df_WP_%s' % year] = globals()['df_WP_%s' % year].reset_index(drop=True)

In [33]:
for year in range(2000,2022):
    globals()['df_WP_%s' % year] =  globals()['df_WP_%s' % year].drop(['SEASON'], axis=1)

In [34]:
for year in range(2000,2022):
    for i in range(len(globals()['df_WP_%s' % year])):
        globals()['df_WP_%s' % year].iloc[i,1] = re.sub(r"[^\w\s]", '',  globals()['df_WP_%s' % year].iloc[i,1])

In [35]:
summarylist = []
for year in range(2000,2022):
    summarylist.append(dict(tuple(globals()['df_WP_%s' % year].groupby('NAME'))))

In [36]:
tc_name_list = []
for i in range(len(summarylist)):
    tc_name_list.append(list(summarylist[i].keys()))

In [37]:
print(tc_name_list[0])

['BEBINCA', 'BILIS', 'BOLAVEN', 'BOPHA', 'CHANCHU', 'DAMREY', 'EWINIAR', 'JELAWAT', 'KAEMI', 'KAITAK', 'KIROGI', 'LONGWANG', 'MARIA', 'PRAPIROON', 'RUMBIA', 'SAOMAI', 'SHANSHAN', 'SONAMU', 'SOULIK', 'TEMBIN', 'WENE', 'WUKONG', 'XANGSANE', 'YAGI']


In [38]:
count = 0
for i in range(len(tc_name_list)):
    for name in tc_name_list[i]:
        count = count+1
        globals()['df_WP_%s_%s'%((str(2000+i)), name)] = globals()['df_WP_%s' % (str(2000+i))][globals()['df_WP_%s' % (str(2000+i))].NAME == name]
#         globals()['df_WP_%s_%s'%((str(2000+i)), name)]  = globals()['df_WP_%s_%s'%((str(2000+i)), name)][globals()['df_WP_%s_%s'%((str(2000+i)),name)].USA_LAT != ' ']
        globals()['df_WP_%s_%s'%((str(2000+i)), name)]  = globals()['df_WP_%s_%s'%((str(2000+i)), name)] .reset_index(drop=True)

In [39]:
# for i in range(len(tc_name_list)):
#     for name in tc_name_list[i]:
#         globals()['df_WP_%s_%s'%((str(2000+i)), name)] = (globals()['df_WP_%s_%s'%((str(2000+i)), name)]).drop(['NAME'], axis=1)

In [40]:
for i in range(len(tc_name_list)):
    for name in tc_name_list[i]:
        globals()['df_WP_%s_%s'%((str(2000+i)), name)].rename(columns={'ISO_TIME':'Date'}, inplace=True)
        globals()['df_WP_%s_%s'%((str(2000+i)), name)]['Date'] = pd.to_datetime(globals()['df_WP_%s_%s'%((str(2000+i)), name)]['Date'], dayfirst=True)

In [41]:
cols_features = list(df_WP_2018_MANGKHUT)[1:len(df_WP_2018_MANGKHUT.columns)]
print(cols_features)

['NAME', 'USA_LAT', 'USA_LON', 'DIST2LAND', 'USA_WIND', 'USA_PRES', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW', 'USA_R34_NW', 'USA_R50_NE', 'USA_R50_SE', 'USA_R50_SW', 'USA_R50_NW', 'USA_R64_NE', 'USA_R64_SE', 'USA_R64_SW', 'USA_R64_NW']


In [42]:
cols_target = list(df_WP_2018_MANGKHUT)[1:2]

In [43]:
cols_target

['NAME']

In [44]:
print(cols_features)
print(cols_target)

['NAME', 'USA_LAT', 'USA_LON', 'DIST2LAND', 'USA_WIND', 'USA_PRES', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW', 'USA_R34_NW', 'USA_R50_NE', 'USA_R50_SE', 'USA_R50_SW', 'USA_R50_NW', 'USA_R64_NE', 'USA_R64_SE', 'USA_R64_SW', 'USA_R64_NW']
['NAME']


In [45]:
# for i in range(len(tc_name_list)):
#     for name in tc_name_list[i]:
#         globals()['df_features_lat_%s_%s'%((str(2000+i)), name)] = globals()['df_WP_%s_%s'%((str(2000+i)), name)][cols_features].astype(float)
#         globals()['df_target_lat_%s_%s'%((str(2000+i)), name)] = globals()['df_WP_%s_%s'%((str(2000+i)), name)][cols_target].astype(float)

In [46]:
df_WP_2000_DAMREY

,Date,NAME,USA_LAT,USA_LON,DIST2LAND,USA_WIND,USA_PRES,USA_R34_NE,USA_R34_SE,USA_R34_SW,USA_R34_NW,USA_R50_NE,USA_R50_SE,USA_R50_SW,USA_R50_NW,USA_R64_NE,USA_R64_SE,USA_R64_SW,USA_R64_NW
0,2000-05-03 18:00:00,DAMREY,0.098000,0.343355,0.243235,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2000-05-03 21:00:00,DAMREY,0.109451,0.342363,0.264317,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000-05-04 00:00:00,DAMREY,0.120000,0.341551,0.281938,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000-05-04 03:00:00,DAMREY,0.128900,0.341070,0.294210,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000-05-04 06:00:00,DAMREY,0.136000,0.340950,0.302077,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2000-05-12 00:00:00,DAMREY,0.528000,0.404089,0.309314,0.181287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,2000-05-12 03:00:00,DAMREY,0.533752,0.410595,0.317810,0.163743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,2000-05-12 06:00:00,DAMREY,0.538000,0.416115,0.328823,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,2000-05-12 09:00:00,DAMREY,0.539744,0.419934,0.340151,0.152047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
tc_name_list[18].remove('MANGKHUT')

In [48]:
tc_name_list[18].remove('KONGREY')

In [49]:
tc_name_list[18].remove('YUTU')

In [50]:
tc_name_list[19].remove('WUTIP')

In [51]:
count = 0

In [52]:
def get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future):
    global count
    features_list = []
    image_list = []
    count_photo =[]
    image_list_current = []
    for i in range(n_past):
        features_list_current = []
        image_list_current = []
        root_dir = "C:/Users/Louis/Full Typhoon Images/"
        # current_typhoon_name = df_WP_2000_DAMREY.iloc[idx,1]
        # current_timestamp_year = (str(df_WP_2000_DAMREY.iloc[idx,0]))[:4]
        folder = "(" + typhoon_name + ")_" + typhoon_year
        current_timestamp_date = (str(globals()['df_WP_%s_%s'% (typhoon_year, typhoon_name)].iloc[idx,0]))[:10]
        current_timestamp_hour = (str(globals()['df_WP_%s_%s'% (typhoon_year, typhoon_name)].iloc[idx,0]))[11:13]
        image_path = "(" + typhoon_name + ")_" + current_timestamp_date + "_" + current_timestamp_hour + ".png"
        img_name = os.path.join(root_dir, folder ,image_path).replace("\\","/")
        # print(img_name)
        try:
            # image_test = io.imread(img_name)
            scaler = transforms.Scale((224, 224))
            # normalize = transforms.Normalize((0.5), (0.5))
            normalize = transforms.Normalize((0.5), (0.5))
            to_tensor = transforms.ToTensor()
            
            image_test = Image.open(img_name)
            image = Variable(normalize(to_tensor(scaler(image_test))).unsqueeze(0))
            
            # image_test = Image.open(img_name).convert('RGB')
            # image_test = np.array(image_test)
            # image_test = np.rollaxis(image_test, 2, 0)
        
            # print(image.shape)
            # current_image_vector = get_vector(img_name)
            # current_image_vector = current_image_vector.numpy()
            # print("OK")
            count = count+1
            # print(count)
            # print(typhoon_name)
            # print(typhoon_year)
            # print("read image")
        except FileNotFoundError:
            # print("FileNotFoundError")
            # print(typhoon_name)
            # print(typhoon_year)
            # print(current_timestamp_date)
            # print("No satelite image for this timestamp")
            # image = "no_image"
            image = torch.zeros(1, 1,224,224)
            # print(image.shape)

        for j in range(17):
            features_list_current.append(globals()['df_WP_%s_%s'% (typhoon_year, typhoon_name)].iloc[idx,j+2])
        image_list_current.append(image)
        
        features_list.append(features_list_current)
        image_list.append(image_list_current)
    
        # features_list = np.array(features_list, dtype=np.float32)
        # image_list = np.array(image_list, dtype=np.float32)
        
        # all_features_numpy = np.concatenate((features_list_current, current_image_vector), axis=0)
        # feature_list.append(all_features_numpy)
        # image_list.append(typhoon_image_current_timestamp)
        # typhoon_dict['features'].append(features_list_current)
        # typhoon_dict['typhoon_image'].append(typhoon_image_current_timestamp)
        idx += 1
    target_latitude = globals()['df_WP_%s_%s'% (typhoon_year, typhoon_name)].iloc[idx,2]
    target_longitude =  globals()['df_WP_%s_%s'% (typhoon_year, typhoon_name)].iloc[idx,3]
    return features_list, image_list, target_latitude, target_longitude,count

In [53]:

all_features_typhoon_data=[]
all_target_typhoon_lat=[]
all_target_typhoon_long=[]
for i in range(len(tc_name_list)):
# for i in range(1):
    for typhoon_name in tc_name_list[i]:
        idx=0
        typhoon_year = str(2000 + i)
        # print(typhoon_year)
        # print(typhoon_name)
        while True:
            try:
                list_1, list_2, lat,long, count = get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future)
                features_tensor = torch.FloatTensor(list_1)
                image_path = list_2
                lat_tensor = torch.from_numpy(np.array(lat, dtype="float32"))
                long_tensor = torch.from_numpy(np.array(long, dtype="float32"))

                
                # print("doing")
                typhoon_features_dict = {'features': features_tensor,
                                'images': image_path}
                
                typhoon_target_lat = {'latitude':lat_tensor}
                typhoon_target_long = {'longitude':long_tensor}

                all_features_typhoon_data.append(typhoon_features_dict)
                all_target_typhoon_lat.append(typhoon_target_lat)
                all_target_typhoon_long.append(typhoon_target_long)
                
                idx += 1
                list_1 = []
                list_2 = []
                lat = 0 
                long = 0
            except IndexError:
                print("no more timestamp")
                break

C:\Users\Louis\anaconda3\lib\site-packages\torchvision\transforms\transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more timestamp
no more ti

In [54]:
all_features_typhoon_data_MANGKHUT=[]
all_target_typhoon_lat_MANGKHUT=[]
all_target_typhoon_long_MANGKHUT=[]

typhoon_name = 'MANGKHUT'
idx=0
typhoon_year = str(2000 + 18)
# print(typhoon_year)
# print(typhoon_name)
while True:
    try:
        list_1, list_2, lat,long, count = get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future)
        features_tensor = torch.FloatTensor(list_1)
        image_path = list_2
        lat_tensor = torch.from_numpy(np.array(lat, dtype="float32"))
        long_tensor = torch.from_numpy(np.array(long, dtype="float32"))


        # print("doing")
        typhoon_features_dict_MANGKHUT = {'features': features_tensor,
                        'images': image_path}

        typhoon_target_lat_MANGKHUT = {'latitude':lat_tensor}
        typhoon_target_long_MANGKHUT = {'longitude':long_tensor}

        all_features_typhoon_data_MANGKHUT.append(typhoon_features_dict_MANGKHUT)
        all_target_typhoon_lat_MANGKHUT.append(typhoon_target_lat_MANGKHUT)
        all_target_typhoon_long_MANGKHUT.append(typhoon_target_long_MANGKHUT)

        idx += 1
        list_1 = []
        list_2 = []
        lat = 0 
        long = 0
    except IndexError:
        print("no more timestamp")
        break

no more timestamp


In [55]:
all_features_typhoon_data_KONGREY=[]
all_target_typhoon_lat_KONGREY=[]
all_target_typhoon_long_KONGREY=[]

typhoon_name = 'KONGREY'
idx=0
typhoon_year = str(2000 + 18)
# print(typhoon_year)
# print(typhoon_name)
while True:
    try:
        list_1, list_2, lat,long, count = get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future)
        features_tensor = torch.FloatTensor(list_1)
        image_path = list_2
        lat_tensor = torch.from_numpy(np.array(lat, dtype="float32"))
        long_tensor = torch.from_numpy(np.array(long, dtype="float32"))


        # print("doing")
        typhoon_features_dict_KONGREY = {'features': features_tensor,
                        'images': image_path}

        typhoon_target_lat_KONGREY = {'latitude':lat_tensor}
        typhoon_target_long_KONGREY = {'longitude':long_tensor}

        all_features_typhoon_data_KONGREY.append(typhoon_features_dict_KONGREY)
        all_target_typhoon_lat_KONGREY.append(typhoon_target_lat_KONGREY)
        all_target_typhoon_long_KONGREY.append(typhoon_target_long_KONGREY)

        idx += 1
        list_1 = []
        list_2 = []
        lat = 0 
        long = 0
    except IndexError:
        print("no more timestamp")
        break

no more timestamp


In [56]:
all_features_typhoon_data_YUTU=[]
all_target_typhoon_lat_YUTU=[]
all_target_typhoon_long_YUTU=[]

typhoon_name = 'YUTU'
idx=0
typhoon_year = str(2000 + 18)
# print(typhoon_year)
# print(typhoon_name)
while True:
    try:
        list_1, list_2, lat,long, count = get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future)
        features_tensor = torch.FloatTensor(list_1)
        image_path = list_2
        lat_tensor = torch.from_numpy(np.array(lat, dtype="float32"))
        long_tensor = torch.from_numpy(np.array(long, dtype="float32"))


        # print("doing")
        typhoon_features_dict_YUTU = {'features': features_tensor,
                        'images': image_path}

        typhoon_target_lat_YUTU = {'latitude':lat_tensor}
        typhoon_target_long_YUTU = {'longitude':long_tensor}

        all_features_typhoon_data_YUTU.append(typhoon_features_dict_YUTU)
        all_target_typhoon_lat_YUTU.append(typhoon_target_lat_YUTU)
        all_target_typhoon_long_YUTU.append(typhoon_target_long_YUTU)

        idx += 1
        list_1 = []
        list_2 = []
        lat = 0 
        long = 0
    except IndexError:
        print("no more timestamp")
        break

no more timestamp


In [57]:
all_features_typhoon_data_WUTIP=[]
all_target_typhoon_lat_WUTIP=[]
all_target_typhoon_long_WUTIP=[]

typhoon_name = 'WUTIP'
idx=0
typhoon_year = str(2000 + 19)
# print(typhoon_year)
# print(typhoon_name)
while True:
    try:
        list_1, list_2, lat,long, count = get_typhoon_data(typhoon_name, typhoon_year, idx , n_past, n_future)
        features_tensor = torch.FloatTensor(list_1)
        image_path = list_2
        lat_tensor = torch.from_numpy(np.array(lat, dtype="float32"))
        long_tensor = torch.from_numpy(np.array(long, dtype="float32"))


        # print("doing")
        typhoon_features_dict_WUTIP = {'features': features_tensor,
                        'images': image_path}

        typhoon_target_lat_WUTIP = {'latitude':lat_tensor}
        typhoon_target_long_WUTIP = {'longitude':long_tensor}

        all_features_typhoon_data_WUTIP.append(typhoon_features_dict_WUTIP)
        all_target_typhoon_lat_WUTIP.append(typhoon_target_lat_WUTIP)
        all_target_typhoon_long_WUTIP.append(typhoon_target_long_WUTIP)

        idx += 1
        list_1 = []
        list_2 = []
        lat = 0 
        long = 0
    except IndexError:
        print("no more timestamp")
        break

no more timestamp


In [58]:
len(all_features_typhoon_data_MANGKHUT)

90

In [59]:
len(all_target_typhoon_lat_MANGKHUT)

90

In [60]:
len(all_target_typhoon_long_MANGKHUT)

90

In [61]:
len(all_features_typhoon_data)

28712

In [62]:
len(all_target_typhoon_lat)

28712

In [63]:
trainX_lat, valX_lat = all_features_typhoon_data[:(int(len(all_features_typhoon_data) * 0.8))], all_features_typhoon_data[(int(len(all_features_typhoon_data) * 0.8)):]
trainY_lat, valY_lat = all_target_typhoon_lat[:(int(len(all_target_typhoon_lat) * 0.8))], all_target_typhoon_lat[(int(len(all_target_typhoon_lat) * 0.8)):]
valX_lat, testX_lat = valX_lat[:(int(len(valX_lat) * 0.5))], valX_lat[(int(len(valX_lat) * 0.5)):]
valY_lat, testY_lat = valY_lat[:(int(len(valY_lat) * 0.5))], valY_lat[(int(len(valY_lat) * 0.5)):]

In [64]:
trainX_long, valX_long = all_features_typhoon_data[:(int(len(all_features_typhoon_data) * 0.8))], all_features_typhoon_data[(int(len(all_features_typhoon_data) * 0.8)):]
trainY_long, valY_long = all_target_typhoon_long[:(int(len(all_target_typhoon_long) * 0.8))], all_target_typhoon_long[(int(len(all_target_typhoon_long) * 0.8)):]
valX_long, testX_long = valX_long[:(int(len(valX_long) * 0.5))], valX_long[(int(len(valX_long) * 0.5)):]
valY_long, testY_long = valY_long[:(int(len(valY_long) * 0.5))], valY_long[(int(len(valY_long) * 0.5)):]

In [65]:
print(len(trainX_long))
print(len(valX_long))
print(len(testX_long))

22969
2871
2872


In [66]:
print(len(trainX_lat))
print(len(valX_lat))
print(len(testX_lat))

22969
2871
2872


In [67]:
train_set_with_lat = [trainX_lat, trainY_lat]

In [68]:
val_set_with_lat = [valX_lat, valY_lat]

In [69]:
test_set_with_lat = [testX_lat, testY_lat]

In [70]:
train_set_with_long = [trainX_long, trainY_long]

In [71]:
val_set_with_long = [valX_long, valY_long]

In [72]:
test_set_with_long = [testX_long, testY_long]

In [73]:
MANGKHUT_set_lat = [all_features_typhoon_data_MANGKHUT, all_target_typhoon_lat_MANGKHUT]
MANGKHUT_set_long = [all_features_typhoon_data_MANGKHUT, all_target_typhoon_long_MANGKHUT]

In [74]:
KONGREY_set_lat = [all_features_typhoon_data_KONGREY, all_target_typhoon_lat_KONGREY]
KONGREY_set_long = [all_features_typhoon_data_KONGREY, all_target_typhoon_long_KONGREY]

In [75]:
YUTU_set_lat = [all_features_typhoon_data_YUTU, all_target_typhoon_lat_YUTU]
YUTU_set_long = [all_features_typhoon_data_YUTU, all_target_typhoon_long_YUTU]

In [76]:
WUTIP_set_lat = [all_features_typhoon_data_WUTIP, all_target_typhoon_lat_WUTIP]
WUTIP_set_long = [all_features_typhoon_data_WUTIP, all_target_typhoon_long_WUTIP]

In [77]:
from torch.utils.data import Dataset,DataLoader
import cv2
import tensorflow as tf

In [78]:
class CustomTensorDataset(Dataset):
  def __init__(self, dataset, transform_list=None):
    [data_X, data_y] = dataset
    #X_tensor, y_tensor = Tensor(data_X), Tensor(data_y)
    tensors = (data_X, data_y)
    assert all(len(tensors[0]) == len(tensor) for tensor in tensors)
    self.tensors = tensors
    
  def __getitem__(self, index):
    x = self.tensors[0][index]
    y = self.tensors[1][index]
    
    tmp_img_data = []
    images_data = []
    
    for image_name in x['images']:
        
        tmp_img_data.append(image_name)
        
    images_data.append(tmp_img_data)

    
    inputs = {
        'features': x['features'],
        'images': images_data
    }
        
    # print(x)
    # print(y)
    
    return inputs, y

  def __len__(self):
    return len(self.tensors[0])

In [79]:
train_set_lstm = CustomTensorDataset(dataset=train_set_with_lat)

In [80]:
val_set_lstm = CustomTensorDataset(dataset=val_set_with_lat)

In [81]:
test_set_lstm = CustomTensorDataset(dataset=test_set_with_lat)

In [82]:
train_set_lstm_long = CustomTensorDataset(dataset=train_set_with_long)

In [83]:
val_set_lstm_long = CustomTensorDataset(dataset=val_set_with_long)

In [84]:
test_set_lstm_long = CustomTensorDataset(dataset=test_set_with_long)

In [85]:
MANGKHUT_set_lstm_lat = CustomTensorDataset(dataset=MANGKHUT_set_lat)
MANGKHUT_set_lstm_long = CustomTensorDataset(dataset=MANGKHUT_set_long)

In [86]:
KONGREY_set_lstm_lat = CustomTensorDataset(dataset=KONGREY_set_lat)
KONGREY_set_lstm_long = CustomTensorDataset(dataset=KONGREY_set_long)

In [87]:
YUTU_set_lstm_lat = CustomTensorDataset(dataset=YUTU_set_lat)
YUTU_set_lstm_long = CustomTensorDataset(dataset=YUTU_set_long)

In [88]:
WUTIP_set_lstm_lat = CustomTensorDataset(dataset=WUTIP_set_lat)
WUTIP_set_lstm_long = CustomTensorDataset(dataset=WUTIP_set_long)

### DataLoader from dictionary

In [89]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim_met, output_dim_met, input_dim_image, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        #fc 17 ->512
        self.fc_feature_met_expansion = nn.Linear(input_dim_met, output_dim_met)
        
        input_dim = output_dim_met + input_dim_image
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # split 529 features into 17 + 512
        # 512 --> 32 dimensions

        #################################### positional encoding
        
    
        # print("x entering forward: ", x)
        # print(x.shape)
        # print(x[0][0][:17])
        
        # print("original x : ", x.shape)
        
        final_feature_list = []
        for i in range(x.shape[0]):
            int_list = []
            for j in range(x.shape[1]):
                up_features_met = self.fc_feature_met_expansion(x[i][j][:17])
                # print(up_features_met.shape)
                # print("upfeatures: ", up_features_met)
                image_feature = x[i][j][17:]
                # print(image_feature.shape)
                # print('image: ',image_feature)
                x_final = torch.cat([up_features_met, image_feature]).to(device)
                # print(x_final.shape)
                int_list.append(x_final)
                #print(int_list)
            x_final = (torch.stack((int_list[0], int_list[1], int_list[2], int_list[3], int_list[4]), 0))
            x_final = x_final[None,:,:]
            #print(x_final)
            # print(x_final.shape)
            final_feature_list.append(x_final)
        
        #print(final_feature_list)
    
        first_feature_tensor = final_feature_list[0]
        for i in range(len(final_feature_list)):
            if i < (len(final_feature_list) - 1):
                final_feature_tensor = torch.cat([first_feature_tensor, final_feature_list[i+1]], dim=0)
                first_feature_tensor = final_feature_tensor
        x_final = final_feature_tensor
        
        #print("final entering ", x_final)

        # print('Final x : ', x_final.shape)
    # def forward(self, x):
        
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x_final.size(0), self.hidden_dim, device=x_final.device).requires_grad_()

        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.layer_dim, x_final.size(0), self.hidden_dim, device=x_final.device).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x_final, (h0.detach(), c0.detach()))
        
        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]
        
        
        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        
        return out

In [90]:
validation_list_long = []

In [91]:
class Optimization_Long:
    def __init__(self, model, loss_fn, optimizer, resnet_model):
        self.model = model
        #self.resnet = image_ex_model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.model_resnet = resnet_model.to(device)
        self.model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.layer = self.model_resnet._modules.get('avgpool')

    def Resnet(self,image):
        # print(t_img)
        # 3. Create a vector of zeros that will hold our feature vector
        #    The 'avgpool' layer has an output size of 512
        my_embedding = torch.zeros(512)
        # 4. Define a function that will copy the output of a layer
        def copy_data(m, i, o):
            my_embedding.copy_(o.data.reshape(o.data.size(1)))
        # 5. Attach that function to our selected layer
        h = self.layer.register_forward_hook(copy_data)
        # 6. Run the model on our transformed image
        self.model_resnet = self.model_resnet.to(device)
        self.model_resnet(image.to(device))
        # 7. Detach our copy function from the layer
        h.remove()
        # 8. Return the feature vector
        return my_embedding
    
    def tensor_concat(self,typhoon_features, typhoon_images, batch):
        
        final_tensor_list = []
        for bat in range(batch):
            # print("Batch size: ", bat)
            image_feature_model = []
            for i in range(len(typhoon_images[0])):
                image_Feature = self.Resnet(typhoon_images[0][i][0][bat].to(device))
                # print("Individual: ", image_Feature)
                image_feature_model.append(image_Feature)

            # print("Group: ", image_feature_model)
            
            image_tensor = torch.stack((image_feature_model[0], image_feature_model[1], image_feature_model[2], image_feature_model[3], image_feature_model[4]), 0)
            image_tensor = image_tensor[None,:,:]
            
            
            # print("image tensor", image_tensor.data[0])       
#             # print(typhoon_features.shape)
            # print('features: ', typhoon_features.data[bat])

            merge_image_and_feature_list = []
            for i in range(len(typhoon_features.data[0])):
                merge_image_and_feature = torch.concat((typhoon_features.data[bat][i].to(device), image_tensor.data[0][i].to(device)), 0)
                # print(merge_image_and_feature.shape)
                merge_image_and_feature_list.append(merge_image_and_feature)

            final_tensor = torch.stack((merge_image_and_feature_list[0], merge_image_and_feature_list[1], merge_image_and_feature_list[2], merge_image_and_feature_list[3], merge_image_and_feature_list[4]), 0)

            final_tensor = final_tensor[None,:,:]
            
            
            # print("final tensor: ", final_tensor)
            # print(final_tensor)
            # print(final_tensor.shape)
            final_tensor_list.append(final_tensor)

        return final_tensor_list
        
    
    def train_step(self, typhoon_features, typhoon_images, y, batch):
        
        
        # print("first feature: ", typhoon_features)
        
        # Sets model to train mode
        self.model.train()
            
        final_tensor_list = self.tensor_concat(typhoon_features,typhoon_images, batch)
                
        y = y[None,:].to(device)
        
        y =torch.transpose(y, 0, 1)
        
        # print("real y :", y.shape)
    
        # Makes predictions                
        first_tensor = final_tensor_list[0]
        for i in range(len(final_tensor_list)):
            if i < (len(final_tensor_list) - 1):
                # print(i)
                final_tensor = torch.cat([first_tensor, final_tensor_list[i+1]], dim=0)
                first_tensor = final_tensor
        
        # print(final_tensor.shape)
        
        final_tensor = final_tensor.to(device)
        
        final_yhat = self.model(final_tensor)
        
        # print("yhat: ", final_yhat.shape)
        
        
#         yhat_list = []
#         for i in range(len(final_tensor_list)):
#             yhat = self.model(final_tensor_list[i].to(device))
#             yhat_list.append(yhat)
        
#         prev_yhat = yhat_list[0]
#         for i in range(len(yhat_list)):
#             if i<(len(yhat_list)-1):
#                 final_yhat = torch.cat((prev_yhat, yhat_list[i+1]), 1)
#                 prev_yhat = final_yhat
            
        
        #Computes loss
        loss = self.loss_fn(y, final_yhat)  #yhat + x0 (lat/long)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()
    
    def train(self, train_loader, val_loader, batch_size, n_epochs):
        # model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        
        model_path = "model_long"
        previous_validation_loss = 100
        patience = 5
        count = 0
        
        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                # print(x_batch['features'])
                # print(x_batch['images'])     
                # print('Y batch',y_batch)
                x_batch = x_batch
                y_batch = y_batch
                x_batch['features'] = x_batch['features'].view([batch_size, n_past, 17])
                
                y_batch = y_batch['longitude']
                
                # x_batch = x_batch.view([batch_size, 5, n_features])
                # print(x_batch['images'])
                
                # print(x_batch)
                # print(y_batch)
                # print("y_batch:", y_batch)
                
                loss = self.train_step(x_batch['features'],x_batch['images'], y_batch, batch_size)
                
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)
            print(training_loss)
            
            
            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val['features'] = x_val['features'].view([batch_size, n_past, 17]).to(device)
                    y_val = y_val['longitude'].to(device)
                    y_val = y_val[None,:].to(device)
                    final_tensor_list_val = self.tensor_concat(x_val['features'],x_val['images'], batch_size)

                    y_val =torch.transpose(y_val, 0, 1)

                    # Makes predictions                
                    first_tensor_val = final_tensor_list_val[0]
                    for i in range(len(final_tensor_list_val)):
                        if i < (len(final_tensor_list_val) - 1):
                            final_tensor_val = torch.cat([first_tensor_val, final_tensor_list_val[i+1]], dim=0)
                            first_tensor_val = final_tensor_val
                    
                            
                    final_tensor_val = final_tensor_val.to(device)
                    
                    self.model.eval()
                    final_yhat_val = self.model(final_tensor_val)
                    
                    val_loss = self.loss_fn(y_val, final_yhat_val).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)
                print(f"[{epoch}/{n_epochs}] Training loss: {training_loss:.8f}\t Validation loss: {validation_loss:.8f}")
                validation_list_long.append(validation_loss)
                validation_list_long.sort()
                if validation_loss <= validation_list_long[0]:
                    torch.save(self.model, model_path)
                    print("Updated best model")
                if validation_loss > previous_validation_loss:
                    count +=1
                    print("Current Validation_loss is greater than Previous Validation Loss, counting:", count)
                    if count>=patience:
                        print("Early stopping executed!")
                        break
                else:
                    print("Continue progress to global minima")
                    count = 0
                previous_validation_loss = validation_loss 

        # torch.save(self.model.state_dict(), model_path) 
        
    
    def evaluate(self, test_loader, batch_size, n_features):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test['features'] = x_test['features'].view([batch_size, n_past, n_features]).to(device)
                y_test = y_test['longitude'].to(device)
                y_test = y_test[None,:].to(device)
                final_tensor_list_test = self.tensor_concat(x_test['features'],x_test['images'], batch_size)
                
                y_test =torch.transpose(y_test, 0, 1)
                
                
                # Makes predictions                
                first_tensor_test = final_tensor_list_test[0]
                for i in range(len(final_tensor_list_test)):
                    if i < (len(final_tensor_list_test) - 1):
                        final_tensor_test = torch.cat([first_tensor_test, final_tensor_list_test[i+1]], dim=0)
                        first_tensor_test = final_tensor_test                    
                            
                final_tensor_test = final_tensor_test.to(device)
                
                self.model.eval()
                yhat_test = self.model(final_tensor_test)
                predictions.append(yhat_test.cpu().detach().numpy())
                values.append(y_test.cpu().detach().numpy())

        return predictions, values
    
    def plot_losses(self):
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [92]:
def get_model(model, model_params):
    models = {"lstm": LSTMModel}
    return models.get(model.lower())(**model_params)

In [93]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

input_dim_met = 17
output_dim_met = 512
input_dim_image = 512 
output_dim = 1
layer_dim = 1
dropout = 0
n_epochs = 50
weight_decay = 1e-6

batch_size = [16]
hidden_dim = [64]
learning_rate = [0.001]
    
for batch in batch_size:
    for hd in hidden_dim:
        for lr in learning_rate:
            print("Batch: ", batch, " ", " Hidden dim: ", hd, " ", "Learning rate: ", lr)
            
            train_loader_long = DataLoader(train_set_lstm_long, batch_size=batch, shuffle=True, drop_last=True)
            val_loader_long = DataLoader(val_set_lstm_long, batch_size=batch, shuffle=True, drop_last=True)
            test_loader_long = DataLoader(test_set_lstm_long, batch_size=batch, shuffle=True, drop_last=True)
            
            MANGKHUT_loader = DataLoader(MANGKHUT_set_lstm_long, batch_size=batch, shuffle=False, drop_last=True)
            KONGREY_loader = DataLoader(KONGREY_set_lstm_long, batch_size=batch, shuffle=False, drop_last=True)
            YUTU_loader = DataLoader(YUTU_set_lstm_long, batch_size=batch, shuffle=False, drop_last=True)
            WUTIP_loader = DataLoader(WUTIP_set_lstm_long, batch_size=batch, shuffle=False, drop_last=True)

            model_params = {'input_dim_met': input_dim_met,
                            'output_dim_met': output_dim_met,
                            'input_dim_image': input_dim_image,
                            'hidden_dim' : hd,
                            'layer_dim' : layer_dim,
                            'output_dim' : output_dim,
                            'dropout_prob' : dropout }

            model = get_model('lstm', model_params).to(device)
            Resnet_model = models.resnet18(pretrained=True).to(device)
            
            loss_fn = nn.MSELoss(reduction="mean")
            optimizer = optim.Adam([{"params": Resnet_model.parameters(), "params": model.parameters()}], lr=lr, weight_decay=weight_decay)
            opt_long = Optimization_Long(model=model.to(device), loss_fn=loss_fn, optimizer=optimizer,resnet_model=Resnet_model.to(device))
            opt_long.train(train_loader_long, val_loader_long, batch, n_epochs)
            predictions, values = opt_long.evaluate(test_loader_long, batch, input_dim_met)
            
            predictions_MANGKHUT, values_MANGKHUT = opt_long.evaluate(MANGKHUT_loader, batch, input_dim_met)
            predictions_KONGREY, values_KONGREY = opt_long.evaluate(KONGREY_loader, batch, input_dim_met)
            predictions_YUTU, values_YUTU = opt_long.evaluate(YUTU_loader, batch, input_dim_met)
            predictions_WUTIP, values_WUTIP = opt_long.evaluate(WUTIP_loader, batch, input_dim_met)
            
            opt_long.plot_losses()

Batch:  16    Hidden dim:  64   Learning rate:  0.001


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Louis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Louis\AppData\Local\Temp/ipykernel_10096/1647775908.py", line 39, in <module>
    model = get_model('lstm', model_params).to(device)
  File "C:\Users\Louis\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 899, in to
    return self._apply(convert)
  File "C:\Users\Louis\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 570, in _apply
    module._apply(fn)
  File "C:\Users\Louis\anaconda3\lib\site-packages\torch\nn\modules\rnn.py", line 189, in _apply
    self.flatten_parameters()
  File "C:\Users\Louis\anaconda3\lib\site-packages\torch\nn\modules\rnn.py", line 175, in flatten_parameters
    torch._cudnn_rnn_flatten_weight(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File

TypeError: object of type 'NoneType' has no len()

In [93]:
train_loader_long = DataLoader(train_set_lstm_long, batch_size=16, shuffle=True, drop_last=True)
val_loader_long = DataLoader(val_set_lstm_long, batch_size=16, shuffle=True, drop_last=True)
test_loader_long = DataLoader(test_set_lstm_long, batch_size=16, shuffle=True, drop_last=True)

MANGKHUT_loader = DataLoader(MANGKHUT_set_lstm_long, batch_size=16, shuffle=False, drop_last=True)
KONGREY_loader = DataLoader(KONGREY_set_lstm_long, batch_size=16, shuffle=False, drop_last=True)
YUTU_loader = DataLoader(YUTU_set_lstm_long, batch_size=16, shuffle=False, drop_last=True)
WUTIP_loader = DataLoader(WUTIP_set_lstm_long, batch_size=16, shuffle=False, drop_last=True)

In [94]:
model_long = torch.load('model_long')

In [95]:
loss_fn = nn.MSELoss(reduction="mean")
Resnet_model = models.resnet18(pretrained=True).to(device)
optimizer = optim.Adam([{"params": Resnet_model.parameters(), "params": model_long.parameters()}], lr=0.001, weight_decay=1e-6)
opt_long = Optimization_Long(model=model_long, loss_fn=loss_fn, optimizer=optimizer,resnet_model=Resnet_model.to(device))


In [96]:
predictions, values = opt_long.evaluate(test_loader_long, 16, 17)

In [97]:
len(predictions)

179

In [98]:
len(values)

179

In [101]:
vals = np.concatenate(values, axis=0).ravel()
preds = np.concatenate(predictions, axis=0).ravel()

In [102]:
vals.shape

(2864,)

In [103]:
for i in range(len(vals)):
    vals[i] = scaler_long.inverse_transform(vals[i].reshape(1, -1))

In [104]:
preds.shape

(2864,)

In [105]:
for i in range(len(preds)):
    preds[i] = scaler_long.inverse_transform(preds[i].reshape(1, -1))

In [106]:
vals.shape

(2864,)

In [107]:
preds.shape

(2864,)

In [108]:
from sklearn.metrics import mean_squared_error
import math

In [109]:
print(mean_squared_error(vals, preds))
print(math.sqrt(mean_squared_error(vals, preds)))

0.7479819
0.8648594717855582


In [110]:
vals_from_long = vals

In [111]:
preds_from_long = preds

In [112]:
%store vals_from_long

Stored 'vals_from_long' (ndarray)


In [113]:
%store preds_from_long

Stored 'preds_from_long' (ndarray)


In [128]:
preds_from_long

array([126.28939, 131.32478, 124.7782 , ..., 125.89428, 142.66652,
       141.8792 ], dtype=float32)

In [114]:
predictions_MANGKHUT, values_MANGKHUT = opt_long.evaluate(MANGKHUT_loader, 16, 17)
predictions_KONGREY, values_KONGREY = opt_long.evaluate(KONGREY_loader, 16, 17)
predictions_YUTU, values_YUTU = opt_long.evaluate(YUTU_loader, 16, 17)
predictions_WUTIP, values_WUTIP = opt_long.evaluate(WUTIP_loader, 16, 17)

In [129]:
def longitude_outcome(value, pred):
    vals = np.concatenate(value, axis=0).ravel()
    preds = np.concatenate(pred, axis=0).ravel()
    for i in range(len(vals)):
        vals[i] = scaler_long.inverse_transform(vals[i].reshape(1, -1))
    for i in range(len(preds)):
        preds[i] = scaler_long.inverse_transform(preds[i].reshape(1, -1))
    return vals,preds

In [130]:
vals_mang_long, pred_mang_long = longitude_outcome(values_MANGKHUT, predictions_MANGKHUT)

In [131]:
print(mean_squared_error(vals_mang_long, pred_mang_long))
print(math.sqrt(mean_squared_error(vals_mang_long, pred_mang_long)))

0.103510655
0.3217307188671014


In [132]:
%store vals_mang_long
%store pred_mang_long

Stored 'vals_mang_long' (ndarray)
Stored 'pred_mang_long' (ndarray)


In [133]:
vals_kong_long, pred_kong_long = longitude_outcome(values_KONGREY, predictions_KONGREY)

In [134]:
print(mean_squared_error(vals_kong_long, pred_kong_long))
print(math.sqrt(mean_squared_error(vals_kong_long, pred_kong_long)))

0.16221581
0.40276024381064185


In [135]:
%store vals_kong_long
%store pred_kong_long

Stored 'vals_kong_long' (ndarray)
Stored 'pred_kong_long' (ndarray)


In [136]:
vals_yutu_long, pred_yutu_long = longitude_outcome(values_YUTU, predictions_YUTU)

In [137]:
print(mean_squared_error(vals_yutu_long, pred_yutu_long))
print(math.sqrt(mean_squared_error(vals_yutu_long, pred_yutu_long)))

0.46250066
0.680074011863089


In [138]:
%store vals_yutu_long
%store pred_yutu_long

Stored 'vals_yutu_long' (ndarray)
Stored 'pred_yutu_long' (ndarray)


In [139]:
vals_wutip_long, pred_wutip_long = longitude_outcome(values_WUTIP, predictions_WUTIP)

In [140]:
print(mean_squared_error(vals_wutip_long, pred_wutip_long))
print(math.sqrt(mean_squared_error(vals_wutip_long, pred_wutip_long)))

0.19874887
0.445812597045075


In [141]:
%store vals_wutip_long
%store pred_wutip_long

Stored 'vals_wutip_long' (ndarray)
Stored 'pred_wutip_long' (ndarray)


In [ ]:
predictions_MANGKHUT, values_MANGKHUT = opt_long.evaluate(MANGKHUT_loader, batch, input_dim_met)
predictions_KONGREY, values_KONGREY = opt_long.evaluate(KONGREY_loader, batch, input_dim_met)
predictions_YUTU, values_YUTU = opt_long.evaluate(YUTU_loader, batch, input_dim_met)
predictions_WUTIP, values_WUTIP = opt_long.evaluate(WUTIP_loader, batch, input_dim_met)

In [107]:
def longitude_outcome(value, pred):
    vals = np.concatenate(value, axis=0).ravel()
    preds = np.concatenate(pred, axis=0).ravel()
    for i in range(len(vals)):
        vals[i] = scaler_long.inverse_transform(vals[i].reshape(1, -1))
    for i in range(len(preds)):
        preds[i] = scaler_long.inverse_transform(preds[i].reshape(1, -1))
    return vals,preds

In [108]:
vals_mang_long, pred_mang_long = longitude_outcome(values_MANGKHUT, predictions_MANGKHUT)

In [109]:
print(mean_squared_error(vals_mang_long, pred_mang_long))
print(math.sqrt(mean_squared_error(vals_mang_long, pred_mang_long)))

0.5254472
0.724877361908821


In [110]:
vals_kong_long, pred_kong_long = longitude_outcome(values_KONGREY, predictions_KONGREY)

In [111]:
print(mean_squared_error(vals_kong_long, pred_kong_long))
print(math.sqrt(mean_squared_error(vals_kong_long, pred_kong_long)))

0.21725883
0.466110315159588


In [112]:
vals_yutu_long, pred_yutu_long = longitude_outcome(values_YUTU, predictions_YUTU)

In [113]:
print(mean_squared_error(vals_yutu_long, pred_yutu_long))
print(math.sqrt(mean_squared_error(vals_yutu_long, pred_yutu_long)))

0.31987748
0.5655771171998624


In [114]:
vals_wutip_long, pred_wutip_long = longitude_outcome(values_WUTIP, predictions_WUTIP)

In [115]:
print(mean_squared_error(vals_wutip_long, pred_wutip_long))
print(math.sqrt(mean_squared_error(vals_wutip_long, pred_wutip_long)))

0.26697165
0.5166929917653329


In [116]:
vals_mang_long.tolist()

[165.9720001220703,
 165.3000030517578,
 164.4429931640625,
 163.60000610351562,
 162.9919891357422,
 162.39999389648438,
 161.6529998779297,
 160.8000030517578,
 159.88699340820312,
 158.89999389648438,
 157.85699462890625,
 156.8000030517578,
 155.76199340820312,
 154.8000030517578,
 154.0080108642578,
 153.1999969482422,
 152.17999267578125,
 151.09999084472656,
 150.07000732421875,
 149.10000610351562,
 148.23500061035156,
 147.39999389648438,
 146.47799682617188,
 145.60000610351562,
 144.87799072265625,
 144.1999969482422,
 143.45700073242188,
 142.6999969482422,
 141.9429931640625,
 141.1999969482422,
 140.47000122070312,
 139.8000030517578,
 139.25,
 138.6999969482422,
 138.0,
 137.3000030517578,
 136.72000122070312,
 136.1999969482422,
 135.71499633789062,
 135.1999969482422,
 134.5800018310547,
 133.89999389648438,
 133.19200134277344,
 132.5,
 131.88499450683594,
 131.3000030517578,
 130.69200134277344,
 130.09999084472656,
 129.5419921875,
 129.0,
 128.44200134277344,
 127.

In [117]:
pred_mang_long.tolist()

[164.9572296142578,
 164.30792236328125,
 163.6926727294922,
 163.3903045654297,
 162.11338806152344,
 161.5274200439453,
 160.71974182128906,
 159.8291473388672,
 159.5810546875,
 158.41946411132812,
 157.5498046875,
 156.2412872314453,
 155.4806365966797,
 154.98341369628906,
 152.9163818359375,
 152.3780059814453,
 151.74977111816406,
 150.9011688232422,
 149.8685302734375,
 148.63681030273438,
 147.9139404296875,
 146.96005249023438,
 146.30056762695312,
 145.13418579101562,
 143.60037231445312,
 143.74612426757812,
 142.7760772705078,
 142.03192138671875,
 141.3844757080078,
 140.40919494628906,
 139.6245880126953,
 139.33123779296875,
 138.6431427001953,
 138.38388061523438,
 137.63385009765625,
 136.8529052734375,
 136.25189208984375,
 135.09803771972656,
 134.98077392578125,
 134.51881408691406,
 134.26022338867188,
 133.69802856445312,
 132.93760681152344,
 132.12130737304688,
 131.42547607421875,
 130.69149780273438,
 130.52378845214844,
 129.7166748046875,
 129.5108184814453

In [118]:
vals_kong_long.tolist()

[150.3800048828125,
 149.60000610351562,
 148.7989959716797,
 148.0,
 147.29400634765625,
 146.5,
 145.5,
 144.1999969482422,
 143.10400390625,
 142.1999969482422,
 141.3820037841797,
 140.6999969482422,
 140.09800720214844,
 139.60000610351562,
 139.18499755859375,
 138.8000030517578,
 138.34300231933594,
 137.89999389648438,
 137.53500366210938,
 137.20001220703125,
 136.84201049804688,
 136.5,
 136.1999969482422,
 135.90000915527344,
 135.56500244140625,
 135.1999969482422,
 134.80799865722656,
 134.40000915527344,
 134.0070037841797,
 133.60000610351562,
 133.14199829101562,
 132.6999969482422,
 132.3419952392578,
 132.0,
 131.614990234375,
 131.1999969482422,
 130.73500061035156,
 130.3000030517578,
 129.97000122070312,
 129.6999969482422,
 129.44200134277344,
 129.20001220703125,
 128.95700073242188,
 128.6999969482422,
 128.39999389648438,
 128.10000610351562,
 127.84199523925781,
 127.5999984741211,
 127.35000610351562,
 127.10000610351562,
 126.8499984741211,
 126.599998474121

In [119]:
pred_kong_long.tolist()

[149.8234405517578,
 148.66371154785156,
 148.67137145996094,
 147.46824645996094,
 146.62908935546875,
 146.2799835205078,
 145.21817016601562,
 144.40435791015625,
 142.71522521972656,
 141.66244506835938,
 141.2237091064453,
 140.2471923828125,
 139.95986938476562,
 139.1456298828125,
 138.65721130371094,
 138.3696746826172,
 137.46437072753906,
 137.51055908203125,
 137.00721740722656,
 136.48341369628906,
 136.54254150390625,
 136.02734375,
 135.88027954101562,
 135.09278869628906,
 135.4301300048828,
 134.6202850341797,
 134.41773986816406,
 134.30517578125,
 133.50254821777344,
 133.339111328125,
 133.1425018310547,
 132.45150756835938,
 132.3412628173828,
 131.61907958984375,
 131.38107299804688,
 131.32623291015625,
 130.78504943847656,
 130.2030029296875,
 129.9090576171875,
 129.56619262695312,
 129.428466796875,
 128.66905212402344,
 129.28985595703125,
 128.3940887451172,
 128.31314086914062,
 128.0523681640625,
 127.77731323242188,
 127.59346008300781,
 127.64514923095703

In [120]:
vals_yutu_long.tolist()

[157.97000122070312,
 157.59999084472656,
 157.17201232910156,
 156.6999969482422,
 156.14300537109375,
 155.60000610351562,
 155.19200134277344,
 154.8000030517578,
 154.31500244140625,
 153.8000030517578,
 153.31500244140625,
 152.8000030517578,
 152.20799255371094,
 151.60000610351562,
 151.02700805664062,
 150.5,
 150.02000427246094,
 149.60000610351562,
 149.25,
 148.90000915527344,
 148.46499633789062,
 148.0,
 147.5500030517578,
 147.09999084472656,
 146.65699768066406,
 146.1999969482422,
 145.71499633789062,
 145.1999969482422,
 144.6429901123047,
 144.10000610351562,
 143.63499450683594,
 143.1999969482422,
 142.74200439453125,
 142.3000030517578,
 141.9219970703125,
 141.5,
 140.92999267578125,
 140.3000030517578,
 139.71499633789062,
 139.09999084472656,
 138.4219970703125,
 137.6999969482422,
 136.95799255371094,
 136.1999969482422,
 135.44200134277344,
 134.6999969482422,
 133.97000122070312,
 133.3000030517578,
 132.72799682617188,
 132.1999969482422,
 131.64199829101562

In [121]:
pred_yutu_long.tolist()

[156.8333740234375,
 156.9591522216797,
 155.79505920410156,
 155.7249298095703,
 155.38250732421875,
 154.77548217773438,
 154.0039825439453,
 154.16921997070312,
 153.55050659179688,
 152.84759521484375,
 152.41603088378906,
 152.1165771484375,
 151.7691192626953,
 150.94480895996094,
 149.97134399414062,
 149.50437927246094,
 149.0648956298828,
 148.44570922851562,
 148.38475036621094,
 148.11441040039062,
 147.62869262695312,
 147.6247100830078,
 146.68299865722656,
 146.02801513671875,
 145.96453857421875,
 145.42543029785156,
 145.43890380859375,
 144.4051971435547,
 144.1787109375,
 143.35707092285156,
 143.4110565185547,
 142.4449005126953,
 142.1562042236328,
 141.7424774169922,
 141.60997009277344,
 141.52886962890625,
 140.84678649902344,
 140.03250122070312,
 138.94998168945312,
 139.14630126953125,
 137.99758911132812,
 137.46963500976562,
 136.68519592285156,
 135.78396606445312,
 135.23651123046875,
 134.60748291015625,
 133.88246154785156,
 132.57737731933594,
 132.2052

In [122]:
vals_wutip_long.tolist()

[159.03500366210938,
 158.40000915527344,
 157.68499755859375,
 157.0,
 156.4429931640625,
 155.90000915527344,
 155.24200439453125,
 154.60000610351562,
 154.0850067138672,
 153.60000610351562,
 153.06500244140625,
 152.5,
 151.88499450683594,
 151.3000030517578,
 150.82000732421875,
 150.40000915527344,
 149.99200439453125,
 149.60000610351562,
 149.1999969482422,
 148.8000030517578,
 148.40699768066406,
 148.0,
 147.56500244140625,
 147.09999084472656,
 146.61500549316406,
 146.10000610351562,
 145.54299926757812,
 145.0,
 144.4969940185547,
 144.10000610351562,
 143.8699951171875,
 143.70001220703125,
 143.4530029296875,
 143.1999969482422,
 142.96400451660156,
 142.89999389648438,
 142.8000030517578,
 142.60499572753906,
 142.40000915527344,
 142.25,
 142.10000610351562,
 141.92300415039062,
 141.6999969482422,
 141.40000915527344,
 141.10000610351562,
 140.8769989013672,
 140.6999969482422,
 140.53500366210938,
 140.39999389648438,
 140.28500366210938,
 140.1999969482422,
 140.14

In [123]:
pred_wutip_long.tolist()

[158.3972625732422,
 157.42955017089844,
 157.0106658935547,
 156.35000610351562,
 155.546875,
 155.00216674804688,
 155.04856872558594,
 153.5323028564453,
 153.5575408935547,
 152.619873046875,
 152.25184631347656,
 152.19735717773438,
 151.40733337402344,
 150.78858947753906,
 150.2987823486328,
 149.7999267578125,
 149.19094848632812,
 148.807373046875,
 148.5088348388672,
 148.2421417236328,
 147.52139282226562,
 146.8705596923828,
 146.83056640625,
 146.63922119140625,
 146.23670959472656,
 145.25955200195312,
 145.00540161132812,
 144.54910278320312,
 144.312744140625,
 143.4198760986328,
 142.9183349609375,
 143.13255310058594,
 142.7505645751953,
 142.3494873046875,
 142.58743286132812,
 141.80528259277344,
 142.23196411132812,
 141.39035034179688,
 141.6286163330078,
 141.50448608398438,
 141.13539123535156,
 140.99591064453125,
 140.99205017089844,
 140.6193084716797,
 140.3452606201172,
 140.29429626464844,
 139.7528839111328,
 140.22901916503906,
 140.10427856445312,
 139.